Students should write a more consise code.
The detailed coding here is for demonstration.

# **Integrate AraVec with Spacy.io**

In [ ]:
!pip install gensim spacy nltk

In [2]:
import gensim
import re
import spacy

In [3]:
# Clean/Normalize Arabic Text
def clean_str(text):
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى","\\",'\n', '\t','&quot;','?','؟','!']
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا","","","","ي","",' ', ' ',' ',' ? ',' ؟ ',' ! ']
    
    #remove tashkeel
    p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(p_tashkeel,"", text)
    
    #remove longation
    p_longation = re.compile(r'(.)\1+')
    subst = r"\1\1"
    text = re.sub(p_longation, subst, text)
    
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    
    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])
    
    #trim    
    text = text.strip()

    return text

In [6]:
from google.colab import files

uploaded = files.upload()

Saving full_uni_cbow_100_wiki.mdl to full_uni_cbow_100_wiki.mdl
Saving full_uni_cbow_100_wiki.mdl.trainables.syn1neg.npy to full_uni_cbow_100_wiki.mdl.trainables.syn1neg.npy
Saving full_uni_cbow_100_wiki.mdl.wv.vectors.npy to full_uni_cbow_100_wiki.mdl.wv.vectors.npy


In [7]:
# load the AraVec model
model = gensim.models.Word2Vec.load("full_uni_cbow_100_wiki.mdl")
print("We've",len(model.wv.index2word),"vocabularies")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


We've 320636 vocabularies


In [8]:
# make a directory called "spacyModel"
%mkdir spacyModel

In [9]:
# export the word2vec fomart to the directory
model.wv.save_word2vec_format("./spacyModel/aravec.txt")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [10]:
# using `gzip` to compress the .txt file
!gzip ./spacyModel/aravec.txt

In [11]:
!python -m spacy  init-model ar spacy.aravec.model --vectors-loc ./spacyModel/aravec.txt.gz

✔ Successfully created model
320636it [00:14, 22728.40it/s]
✔ Loaded vectors from spacyModel/aravec.txt.gz
✔ Sucessfully compiled vocab
320826 entries, 320636 vectors


In [12]:
# load AraVec Spacy model
nlp = spacy.load("./spacy.aravec.model/")

In [14]:
# Define the preprocessing Class
class Preprocessor:
    def __init__(self, tokenizer, **cfg):
        self.tokenizer = tokenizer

    def __call__(self, text):
        preprocessed = clean_str(text)
        return self.tokenizer(preprocessed)

In [15]:
#Apply the `Preprocessor` Class
nlp.tokenizer = Preprocessor(nlp.tokenizer)

In [16]:
# Test your model
nlp("قطة").vector

array([-0.5017682 , -0.17910735,  1.3386917 , -0.8353952 , -1.1803102 ,
        0.18786961, -0.82984304,  0.13241217, -1.8108876 , -1.2894193 ,
       -0.5938235 , -0.3651574 ,  0.42976248, -0.42864415, -2.478931  ,
       -0.52788997,  0.1358112 , -1.0793025 ,  0.45681792,  0.18629487,
       -1.5901515 ,  2.2574892 , -1.0637302 ,  0.8920622 , -0.2925793 ,
       -1.0582981 ,  0.08127645,  1.31154   ,  2.1379976 , -0.8491325 ,
        0.8019985 ,  1.7410465 , -0.77410066, -1.9521624 ,  0.6022742 ,
       -2.5095665 , -1.253059  , -1.9257681 , -0.20949468, -0.21627091,
        0.38716242,  1.0295613 ,  3.4647188 , -0.22649226, -0.4813386 ,
        0.5222677 , -0.5111857 ,  0.05956047,  0.14050284, -0.04297049,
        0.78426254, -0.70871925,  1.5848184 , -1.51491   , -0.86398554,
        0.38917732, -1.9502006 , -1.1425861 ,  1.9147727 ,  0.5576095 ,
        0.8455574 ,  0.19763526,  0.160413  ,  0.8102146 ,  1.4172752 ,
       -0.9214981 ,  2.3475652 , -0.53736365, -2.5405118 , -1.14

In [17]:
egypt = nlp("مصر")
tunisia = nlp("تونس")
apple = nlp("تفاح")

print("egypt Vs. tunisia = ", egypt.similarity(tunisia))
print("egypt Vs. apple = ", egypt.similarity(apple))

egypt Vs. tunisia =  0.5449278431882557
egypt Vs. apple =  -0.15080560126990902


# **Spam Detection Using Logistic Regression and Aravec**

In [18]:
import csv
from io import open
import pandas as pd
from sklearn import preprocessing
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.svm import LinearSVC

In [19]:
# load files into the environment
from google.colab import files
uploaded = files.upload()

Saving dataall.xlsx to dataall.xlsx


In [21]:
df = pd.read_excel("dataall.xlsx", usecols=lambda col: col not in ["id","Unnamed: 0","Unnamed: 0.1","Unnamed: 0.2"])
print (df['tweet'])
df = df.drop_duplicates()

# print the number of data
print(len(df))



0       سنين نعاني وبعد ما شفنا النعمه مع الاتصالات ال...
1       #فايبر_stc_يفرق صاحب هذه الفاتوره تنازل عن سته...
2       الف ميغا سرعه الانترنت شهريا في بريطانيا بسعر ...
3       وهذا سعرها الانترنت في بريطانيا والسرعه بالغيغ...
4       #طيف_القحطاني #الراتب #فايبر_stc_يفرق حسبي اله...
                              ...                        
6985            زي كذا يعني #stc5G pictwitercomRQRLRCLCOE
6986     جبتها صح يلا اخذوني  #stc5G pictwitercomuK5El...
6987     #STC5G يلا متي تحجزولي من الرياض  pictwiterco...
6988                  الاول #stc5G pictwitercom1ZHsFG0SYv
6989     اذا قدرت توقفها في المكان المخص  تروح معنا ان...
Name: tweet, Length: 6990, dtype: object
6786


In [22]:
le = preprocessing.LabelEncoder()
y_enc = le.fit_transform(df['class'])

In [23]:
x = np.array([nlp(d).vector for d in df['tweet']])
y = np.array([d for d in y_enc])

# print the dimension of x
print(x.shape)

# print the dimension of y
print(y.shape)

# double check
print(nlp(df['tweet'][0]).text, y[0])


(6786, 100)
(6786,)
سنين نعاني وبعد ما شفنا النعمه مع الاتصالات المتكامله تبون نرجع لكم واله لو بلاش #فايبر stc يفرق 0


In [24]:
X_train,X_test,y_train,y_test = train_test_split(x,y, test_size = 0.2, random_state = 10)

In [25]:
logreg = linear_model.LogisticRegression()
logreg.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [26]:
prediction = dict()
prediction["lr"] = logreg.predict(X_test)
print(prediction["lr"])

[0 1 0 ... 0 1 0]


In [27]:
print(classification_report(y_test,prediction["lr"]))

              precision    recall  f1-score   support

           0       0.78      0.88      0.83       832
           1       0.77      0.61      0.68       526

    accuracy                           0.78      1358
   macro avg       0.78      0.75      0.76      1358
weighted avg       0.78      0.78      0.77      1358



In [28]:
# predict using trained model
predict = logreg.predict(np.array([nlp('خدمة ممتازة').vector, nlp('٣ شهور').vector]))
print(predict)

[0 0]


In [31]:
df.head()

,class,tweet,hashtag,url,number
0,not,سنين نعاني وبعد ما شفنا النعمه مع الاتصالات ال...,1,0,0
1,spam,#فايبر_stc_يفرق صاحب هذه الفاتوره تنازل عن سته...,1,1,0
2,not,الف ميغا سرعه الانترنت شهريا في بريطانيا بسعر ...,1,0,0
3,not,وهذا سعرها الانترنت في بريطانيا والسرعه بالغيغ...,1,1,0
4,spam,#طيف_القحطاني #الراتب #فايبر_stc_يفرق حسبي اله...,3,0,0


# **Spam Detection Using Logistic Regression and Aravec + Other Features**

In [32]:
features=df.loc[:, ["hashtag", "url", "number"]].values
tweets= np.concatenate([x, features], axis=1)

In [33]:
X_train,X_test,y_train,y_test = train_test_split(tweets,y, test_size = 0.2, random_state = 10)

In [34]:
logreg2 = linear_model.LogisticRegression()
logreg2.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [35]:
prediction = dict()
prediction["lr2"] = logreg2.predict(X_test)
print(prediction["lr2"])

[0 0 0 ... 1 0 0]


In [36]:
print(classification_report(y_test,prediction["lr2"]))

              precision    recall  f1-score   support

           0       0.82      0.92      0.86       832
           1       0.84      0.67      0.75       526

    accuracy                           0.82      1358
   macro avg       0.83      0.80      0.81      1358
weighted avg       0.83      0.82      0.82      1358



In [37]:
# predict using trained model
#predict = logreg2.predict(np.array([nlp('خدمة ممتازة').vector, nlp('٣ شهور').vector]))
#print(predict)

# **Spam Detection Using SVM and Aravec**

In [38]:
X_train,X_test,y_train,y_test = train_test_split(x,y, test_size = 0.2, random_state = 10)

In [39]:
svm = LinearSVC().fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [40]:
prediction = dict()
prediction["svm"] = svm.predict(X_test)
print(prediction["svm"])

[0 1 0 ... 0 1 0]


In [41]:
print(classification_report(y_test,prediction["svm"]))

              precision    recall  f1-score   support

           0       0.79      0.89      0.83       832
           1       0.78      0.62      0.69       526

    accuracy                           0.78      1358
   macro avg       0.78      0.75      0.76      1358
weighted avg       0.78      0.78      0.78      1358



# **Spam Detection Using SVM and Aravec + Other Features**

In [42]:
X_train,X_test,y_train,y_test = train_test_split(tweets,y, test_size = 0.2, random_state = 10)

In [43]:
svm2 = LinearSVC().fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [44]:
prediction = dict()
prediction["svm2"] = svm2.predict(X_test)
print(prediction["svm2"])

[0 0 0 ... 1 0 0]


In [45]:
print(classification_report(y_test,prediction["svm2"]))

              precision    recall  f1-score   support

           0       0.81      0.92      0.86       832
           1       0.84      0.67      0.75       526

    accuracy                           0.82      1358
   macro avg       0.83      0.79      0.81      1358
weighted avg       0.83      0.82      0.82      1358

